In [17]:
# ffmpeg: Command Line Interface
# 동영상 압출, 썸네일 얻기, 오디오 추출

import subprocess
from pydub import AudioSegment
import math

# ffmpeg -i ./files/zzoneddeock.mp4 -vn ./files/audio.mp3 -> 커맨드로도 가능
# video_path: 동영상의 경로, audio_path: 추출한 audio가 저장될 경로
def extract_audio_from_video(video_path, audio_path):
    command = ['ffmpeg', '-i', video_path, '-vn', audio_path,]
    subprocess.run(command)

# extract_audio_from_video('../files/100minute.mp4', '../files/100audio.mp3')


def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder, name):
    track = AudioSegment.from_mp3(audio_path)
    # track.duration_seconds # 몇 초인지 반환
    chunk_len = chunk_size * 60 * 1000 # 첫 시간 선택, *1000: pydub은 밀리초단위로 작동
    chunks = math.ceil(len(track) / chunk_len)
    
    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i+1) * chunk_len
        
        chunk = track[start_time:end_time]
        
        chunk.export(f"{chunks_folder}/{name}_{i}.mp3", format="mp3")

In [18]:
cut_audio_in_chunks('../files/100audio.mp3', 10, '../files/chunks', "100minute" )

In [28]:
from openai import OpenAI
import glob # 패턴을 사용해 디렉토리 내부의 파일 검색

client = OpenAI()

def transcribe_chunks(chunk_folder, destination):
    # destination: text file 저장될 위치
    files = glob.glob(f"{chunk_folder}/*.mp3")
    for file in files:
        with open(file, "rb") as audio_file, open(destination, "a", encoding="UTF-8") as text_file:
            transcript = client.audio.transcriptions.create(model="whisper-1", file=audio_file,)
            print(transcript)
            text_file.write(transcript.text)

    
# transcript = openai.Audio.transcribe("whisper-1", open('../files/chunks/100minute_0.mp3', "rb"))
# transcript
# json object가 생성 -> text를 key로 가는다
# transcript['text']

transcribe_chunks("../files/chunks", "../files/transcript.txt")

APIStatusError: Error code: 413 - {'error': {'message': 'Maximum content size limit (26214400) exceeded (26457306 bytes read)', 'type': 'server_error', 'param': None, 'code': None}}

In [1]:
# ffmpeg 실행 파일이 시스템의 전역 PATH에 포함되어 있음에도 불구하고 가상 환경이 그 경로를 상속하지 않아 발생하는 문제
# 가상 환경은 종종 자체적인 격리된 환경을 만들기 때문에, 시스템 전역에 설치된 프로그램에 대한 접근이 제한될 수 있다.
# => 가상 환경 내에서 직접 ffmpeg의 경로를 지정 / Python 코드 내에서 ffmpeg의 전체 경로를 명시적으로 사용
# which ffmpeg

import subprocess

def extract_audio_from_video(video_path, audio_path):
    command = ['/opt/homebrew/bin/ffmpeg', '-i', video_path, '-vn', audio_path]
    subprocess.run(command)
    
extract_audio_from_video("www.ssyoutube.com/watch?v=jlbs5zfSucY", "../files/100audio.mp3")
    

ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvino --e